In [1]:
# 1. Gerekli Kütüphaneleri Yükle (TPU ortamında bazen eksik olabilir)
!pip install transformers accelerate -q

import os
import cv2
import torch
import numpy as np
import requests
from PIL import Image
from google.colab import files
from sklearn.cluster import KMeans
from torchvision import transforms
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# --- TPU İÇİN ÖZEL KISIM BAŞLANGICI ---
# CUDA yerine XLA kütüphanesini kullanmaya çalışıyoruz
try:
    import torch_xla.core.xla_model as xm
    device = xm.xla_device()
    print(f"✅ TPU Başarıyla Aktif Edildi! Cihaz: {device}")
except ImportError:
    print("⚠️ TPU kütüphanesi (torch_xla) bulunamadı. CPU kullanılıyor.")
    device = "cpu"
# --- TPU İÇİN ÖZEL KISIM BİTİŞİ ---

# 3. Video Yükleme Kontrolü
video_path = "/content/video.mp4"
if not os.path.exists(video_path):
    print("Video bulunamadı, lütfen yükleyiniz:")
    uploaded = files.upload()

# --- MODEL YÜKLEME ---
print("DINOv2 Modeli yükleniyor...")
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
dinov2.to(device)
dinov2.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# --- VİDEO OKUMA ---
cap = cv2.VideoCapture(video_path)
frames = []
frame_ids = []
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # HIZLANDIRMA: Her 30. kareyi al
    if frame_count % 30 == 0:
        frames.append(frame)
        frame_ids.append(frame_count)
    frame_count += 1
cap.release()

if len(frames) == 0:
    raise ValueError("Hiç kare okunamadı!")

# --- EMBEDDING ÇIKARMA (TPU KULLANILACAK) ---
print(f"TPU üzerinde {len(frames)} kare işleniyor...")
embeddings = []

for i, frame in enumerate(frames):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Tensörü TPU cihazına gönderiyoruz (.to(device))
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = dinov2(image_tensor)

        # TPU'dan sonucu geri alırken CPU'ya çekmemiz lazım (.cpu())
        embeddings.append(embedding.cpu().numpy().reshape(-1))

    if i % 10 == 0:
        # TPU işlemlerini senkronize etmek bazen gerekebilir
        xm.mark_step()

embeddings = np.array(embeddings)

# --- KÜMELEME ---
NUM_CLUSTERS = min(5, len(frames))
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
labels = kmeans.fit_predict(embeddings)
keyframe_indices = []

for cluster_id in range(NUM_CLUSTERS):
    cluster_indices = np.where(labels == cluster_id)[0]
    cluster_embeddings = embeddings[cluster_indices]
    center = kmeans.cluster_centers_[cluster_id]
    distances = np.linalg.norm(cluster_embeddings - center, axis=1)
    best_local_idx = np.argmin(distances)
    keyframe_indices.append(cluster_indices[best_local_idx])

keyframe_indices = sorted(keyframe_indices)

# --- CAPTIONING ---
# Belleği boşalt
del dinov2
# TPU belleğini temizlemek biraz farklıdır ama garbage collector yeterli olabilir
import gc
gc.collect()

print("BLIP-2 Modeli yükleniyor...")
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

# Modeli TPU'ya atıyoruz
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device)

print("\n--- SONUÇLAR ---")
for idx in keyframe_indices:
    frame = frames[idx]
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    inputs = processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=30)

    caption = processor.decode(output[0], skip_special_tokens=True)
    print(f"Frame {frame_ids[idx]}: {caption}")

ModuleNotFoundError: No module named 'cv2'

In [9]:
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 61.4 MB/s eta 0:00:00


In [10]:
import cv2
import torch
import numpy as np
import os
from sklearn.cluster import KMeans
import gc
from PIL import Image
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from google.colab import files

# 1. Cihazı Belirle (H100 de bir "cuda" cihazıdır)
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Hangi GPU'nun çalıştığını kontrol et
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 KULLANILAN GPU: {gpu_name}")

    # H100 için Hızlandırma Ayarları
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("✅ H100 için TF32 (TensorFloat-32) modu açıldı!")
else:
    print("⚠️ DİKKAT: GPU bulunamadı! Ayarlardan GPU seçtiğine emin misin?")

# 3. Video Yükleme Kontrolü
video_path = "/content/video.mp4"
if not os.path.exists(video_path):
    print("Video bulunamadı, lütfen yükleyiniz:")
    uploaded = files.upload()

# 4. Model Yükleme (Small Model - Bellek Dostu)
print("DINOv2 (Small) Modeli Yükleniyor...")
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2.to(device)
dinov2.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# 5. Video Okuma ve Örnekleme (Sampling)
cap = cv2.VideoCapture(video_path)
frames = []
frame_ids = []
frame_count = 0
SKIP_FRAMES = 30  # Her 30 karede bir örnek al (Yaklaşık 1 FPS)

print("Video okunuyor ve örnekleniyor (Her 30. kare)...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % SKIP_FRAMES == 0:
        frames.append(frame)
        frame_ids.append(frame_count)
    frame_count += 1

cap.release()
print(f"Toplam Kare: {frame_count}, İşlenecek Kare Sayısı: {len(frames)}")

if len(frames) == 0:
    raise ValueError("Video okunamadı!")

# 6. Embedding Çıkarma
embeddings = []
print("Embeddingler çıkarılıyor...")

for i, frame in enumerate(frames):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = dinov2(image_tensor)
        # GPU sonucunu CPU'ya alıp numpy array'e çeviriyoruz
        embeddings.append(embedding.cpu().numpy().reshape(-1))

embeddings = np.array(embeddings)
print(f"Embedding Shape: {embeddings.shape}")

# 7. Kümeleme (Keyframe Seçimi)
NUM_CLUSTERS = min(10, len(frames))
print(f"{NUM_CLUSTERS} adet anahtar kare seçiliyor...")

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
labels = kmeans.fit_predict(embeddings)
keyframe_indices = []

for cluster_id in range(NUM_CLUSTERS):
    cluster_indices = np.where(labels == cluster_id)[0]
    cluster_embeddings = embeddings[cluster_indices]
    center = kmeans.cluster_centers_[cluster_id]
    distances = np.linalg.norm(cluster_embeddings - center, axis=1)
    best = cluster_indices[np.argmin(distances)]
    keyframe_indices.append(best)

keyframe_indices = sorted(keyframe_indices)

# 8. Bellek Temizliği (Captioning Öncesi)
del dinov2
del embeddings
gc.collect()
torch.cuda.empty_cache()  # GPU belleğini temizle

# 9. Captioning (Hafif Model ile)
print("Caption Modeli (BLIP-Large) Yükleniyor...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

print("\n--- SONUÇLAR ---")
for idx in keyframe_indices:
    frame = frames[idx]
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    inputs = processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50)

    caption = processor.decode(output[0], skip_special_tokens=True)
    print(f"Frame {frame_ids[idx]}: {caption}")

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


⚠️ DİKKAT: GPU bulunamadı! Ayarlardan GPU seçtiğine emin misin?
Video bulunamadı, lütfen yükleyiniz:


Saving video.mp4 to video.mp4
DINOv2 (Small) Modeli Yükleniyor...
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip


/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth


...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]


--- SONUÇLAR ---
Frame 480: arafed building with a sign that says surve grosy bye boy bab bab iridid
Frame 1410: there is a large green object on the ground next to a pile of trash
Frame 2370: there is a picture of a chair that has a yellow apple on it
Frame 2970: there is a bathroom with a toilet and a sink in it
Frame 3930: there is a television screen showing a television with a bunch of boxes
Frame 4440: a close up of a chair and a bag on the ground
Frame 4860: there is a man walking down a hallway with a dog
Frame 5700: a close up of a person standing on a stair case
Frame 9180: araf yassim hasi hasi tit haberde
Frame 10500: a close up of a hallway with a bed and a table


In [ ]:
import cv2
import torch
import numpy as np
import os
from sklearn.cluster import KMeans
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from google.colab import files

# --- AYARLAR ---
SKIP_FRAMES = 30       # Her 30 karede bir işlem yap (Yaklaşık 1 FPS)
BLUR_THRESHOLD = 100.0 # Bulanıklık eşiği (Düşük değerler daha bulanık kabul edilir)
MIN_CLUSTERS = 5       # En az kaç keyframe olsun
MAX_CLUSTERS = 20      # En fazla kaç keyframe olsun

# 1. Cihazı Belirle
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Hangi GPU'nun çalıştığını kontrol et
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 KULLANILAN GPU: {gpu_name}")

    # H100 vb. için Hızlandırma Ayarları
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
else:
    print("⚠️ DİKKAT: GPU bulunamadı!")

# 3. Model Yükleme (DINOv2 - Large)
print("DINOv2 (Large) Modeli Yükleniyor...")
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
dinov2.to(device)
dinov2.eval()

# 4. Video İşleme
video_path = "/content/video.mp4"
if not os.path.exists(video_path):
    print("Video bulunamadı, lütfen yükleyiniz:")
    uploaded = files.upload()

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0: fps = 30 # Hata önleyici

frames = []
frame_ids = []
frame_count = 0
valid_frame_count = 0

print("Video taranıyor (Örnekleme + Bulanıklık Kontrolü)...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Adım: Örnekleme (Sadece belirli aralıklarla kare al)
    if frame_count % SKIP_FRAMES == 0:
        # 2. Adım: Bulanıklık Kontrolü
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

        if laplacian_var > BLUR_THRESHOLD:
            frames.append(frame)
            frame_ids.append(frame_count)
            valid_frame_count += 1

    frame_count += 1

cap.release()
print(f"Toplam Kare: {frame_count}")
print(f"Seçilen Net Kare Sayısı: {len(frames)}")

if len(frames) == 0:
    raise ValueError("Hiç uygun kare bulunamadı! Eşik değerini düşürmeyi deneyin.")

# 5. Embedding Çıkarma
embeddings = []
print("Embeddingler çıkarılıyor...")

for frame in frames:
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # DINOv2 için resize
    image = image.resize((224, 224))
    image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = dinov2(image_tensor)

    embeddings.append(embedding.cpu().numpy()[0])

embeddings = np.array(embeddings)

# 6. Dinamik Kümeleme (Video süresine göre)
# Örnek: Her 10 saniye için 1 keyframe, ama min 5 max 20 arasında tut.
duration_sec = frame_count / fps
dynamic_k = int(duration_sec / 10)
NUM_CLUSTERS = max(MIN_CLUSTERS, min(dynamic_k, MAX_CLUSTERS, len(frames)))

print(f"Video Süresi: {duration_sec:.1f}sn -> Hedeflenen Keyframe Sayısı: {NUM_CLUSTERS}")

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
labels = kmeans.fit_predict(embeddings)
keyframe_indices = []

for cluster_id in range(NUM_CLUSTERS):
    cluster_indices = np.where(labels == cluster_id)[0]
    cluster_embeddings = embeddings[cluster_indices]
    center = kmeans.cluster_centers_[cluster_id]
    distances = np.linalg.norm(cluster_embeddings - center, axis=1)
    best = cluster_indices[np.argmin(distances)]
    keyframe_indices.append(best)

keyframe_indices = sorted(keyframe_indices)

# 7. Captioning (BLIP-2)
print("BLIP-2 Modeli Yükleniyor...")
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device)

print("\n--- SONUÇLAR ---")
for idx in keyframe_indices:
    frame = frames[idx]
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    inputs = processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=30)

    caption = processor.decode(output[0], skip_special_tokens=True)
    print(f"Frame {frame_ids[idx]} ({frame_ids[idx]/fps:.1f}s): {caption}")

🚀 KULLANILAN GPU: NVIDIA H100 80GB HBM3
DINOv2 (Large) Modeli Yükleniyor...


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Video taranıyor (Örnekleme + Bulanıklık Kontrolü)...
Toplam Kare: 11087
Seçilen Net Kare Sayısı: 370
Embeddingler çıkarılıyor...
Video Süresi: 443.5sn -> Hedeflenen Keyframe Sayısı: 20
BLIP-2 Modeli Yükleniyor...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


--- SONUÇLAR ---
Frame 240 (9.6s): turkey's president says he will not stop fighting in syria

Frame 390 (15.6s): syrias al-quds hospital in the city of aleppo was destroyed by the regime

Frame 1050 (42.0s): a news screen showing the aftermath of a bombing

Frame 1410 (56.4s): a television screen showing a picture of a gun and a bag

Frame 2370 (94.8s): a television screen shows a chair with a yellow object on it

Frame 2820 (112.8s): a television screen showing a bathroom with a toilet and trash

Frame 3060 (122.4s): a television screen showing a room with a chair and a table

Frame 3870 (154.8s): a television screen showing a room with boxes and other items

Frame 4200 (168.0s): a man is walking through a hallway with a television screen

Frame 6090 (243.6s): a tv screen showing a man walking down a hallway

Frame 6240 (249.6s): a man in a black hoodie is standing in a hallway

Frame 6720 (268.8s): a man in black jacket and hoodie is holding a gun

Frame 6960 (278.4s): a television